In [1]:
import tensorflow.compat.v1 as tf
import numpy as np
import random
import math
import os

tf.compat.v1.disable_eager_execution()

In [2]:
# 학습에 필요한 설정값을 정의한다.

epsilon = 1                 		# epsilon-Greedy 기법에 사용할 최초의 epsilon값
epsilonMinimumValue = 0.001 		# epsilon의 최소값 (이 값 이하로 Decay하지 않습니다)
num_actions = 3               	# 에이전트가 취할 수 있는 행동의 개수 - (좌로 움직이기, 가만히 있기, 우로 움직이기)
num_epochs = 2000 					    # 학습에 사용할 반복횟수
hidden_size = 128 					    # 히든레이어의 노드 개수
maxMemory = 500 					      # Replay Memory의 크기
batch_size = 50 					      # 학습에 사용할 배치 개수
gridSize = 10 						      # 에이전트가 플레이하는 게임 화면 크기 (10x10 grid)
state_size = gridSize * gridSize 	# 게임 환경의 현재상태 (10x10 grid)
discount = 0.9 						        # Discount Factor \gamma
learning_rate = 0.2					      # 러닝 레이트

In [3]:
# 시작과 끝값을 기준으로 랜덤값을 추출하는 함수를 정의한다.
def randf(s, e):
  return (float(random.randrange(0, (e - s) * 9999)) / 10000) + s;

In [4]:
# DQN 모델을 정의합니다.
# 100(현재 상태 - 10x10 Grid) -> 128 -> 128 -> 3(예측된 각 행동의 Q값)
def build_DQN(x):
  W1 = tf.Variable(tf.truncated_normal(shape=[state_size, hidden_size], stddev=1.0 / math.sqrt(float(state_size))))
  b1 = tf.Variable(tf.truncated_normal(shape=[hidden_size], stddev=0.01))  
  H1_output = tf.nn.relu(tf.matmul(x, W1) + b1)
  W2 = tf.Variable(tf.truncated_normal(shape=[hidden_size, hidden_size],stddev=1.0 / math.sqrt(float(hidden_size))))
  b2 = tf.Variable(tf.truncated_normal(shape=[hidden_size], stddev=0.01))
  H2_output = tf.nn.relu(tf.matmul(H1_output, W2) + b2)
  W3 = tf.Variable(tf.truncated_normal(shape=[hidden_size, num_actions],stddev=1.0 / math.sqrt(float(hidden_size))))
  b3 = tf.Variable(tf.truncated_normal(shape=[num_actions], stddev=0.01))
  output_layer = tf.matmul(H2_output, W3) + b3

  return tf.squeeze(output_layer)

In [5]:
# 인풋 화면 이미지와 타겟 Q값을 받기 위한 플레이스 홀더를 선언합니다.
x = tf.placeholder(tf.float32, shape=[None, state_size])
y = tf.placeholder(tf.float32, shape=[None, num_actions])

In [6]:
# DQN 모델을 선언하고 예측결과를 리턴받습니다.
y_pred = build_DQN(x)

In [7]:
# MSE 손실 함수와 옵티마이저를 정의합니다.
loss = tf.reduce_sum(tf.square(y-y_pred)) / (2*batch_size)  # MSE 손실 함수
optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)

In [8]:
# CatchGame을 수행하는 Environment를 구현합니다.
class CatchEnvironment():
  # 상태의 초기값을 지정합니다.
  def __init__(self, gridSize):
    self.gridSize = gridSize
    self.state_size = self.gridSize * self.gridSize
    self.state = np.empty(3, dtype = np.uint8) 

  # 관찰 결과를 리턴합니다.
  def observe(self):
    canvas = self.drawState()
    canvas = np.reshape(canvas, (-1,self.state_size))
    return canvas

  # 현재 상태(fruit, basket)를 화면에 출력합니다.
  def drawState(self):
    canvas = np.zeros((self.gridSize, self.gridSize))
    # fruit를 화면에 그립니다.
    canvas[self.state[0]-1, self.state[1]-1] = 1  
    # basket을 화면에 그립니다. 
    canvas[self.gridSize-1, self.state[2] -1 - 1] = 1
    canvas[self.gridSize-1, self.state[2] -1] = 1
    canvas[self.gridSize-1, self.state[2] -1 + 1] = 1    
    return canvas        

  # 게임을 초기 상태로 리셋합니다.
  def reset(self): 
    initialFruitColumn = random.randrange(1, self.gridSize + 1)
    initialBucketPosition = random.randrange(2, self.gridSize + 1 - 1)
    self.state = np.array([1, initialFruitColumn, initialBucketPosition]) 
    return self.getState()

  # 현재 상태를 불러옵니다.
  def getState(self):
    stateInfo = self.state
    fruit_row = stateInfo[0]
    fruit_col = stateInfo[1]
    basket = stateInfo[2]
    return fruit_row, fruit_col, basket

  # 에이전트가 취한 행동에 대한 보상을 줍니다.
  def getReward(self):
    fruitRow, fruitColumn, basket = self.getState()
    # 만약 fruit가 바닥에 닿았을 때
    if (fruitRow == self.gridSize - 1):  
      # basket이 fruit을 받아내면 1의 reward를 줍니다.
      if (abs(fruitColumn - basket) <= 1): 
        return 1
      # fruit를 받아내지 못하면 -1의 reward를 줍니다.
      else:
        return -1
    # fruit가 바닥에 닿지 않은 중립적인 상태는 0의 reward를 줍니다.
    else:
      return 0

  # 게임이 끝났는지를 체크합니다.(fruit가 바닥에 닿으면 한게임이 종료됩니다.)
  def isGameOver(self):
    if (self.state[0] == self.gridSize - 1): 
      return True 
    else: 
      return False 

  # action(좌로 한칸 이동, 제자리, 우로 한칸이동)에 따라 basket의 위치를 업데이트합니다.
  def updateState(self, action):
    move = 0
    if (action == 0):
      move = -1
    elif (action == 1):
      move = 0
    elif (action == 2):
      move = 1
    fruitRow, fruitColumn, basket = self.getState()
    newBasket = min(max(2, basket + move), self.gridSize - 1) # min/max는 basket이 grid밖으로 벗어나는것을 방지합니다.
    fruitRow = fruitRow + 1  # fruit는 매 행동을 취할때마다 1칸씩 아래로 떨어집니다. 
    self.state = np.array([fruitRow, fruitColumn, newBasket])

  # 행동을 취합니다. 0 : 왼쪽으로 이동, 1 : 가만히 있기, 2 : 오른쪽으로 이동
  def act(self, action):
    self.updateState(action)
    reward = self.getReward()
    gameOver = self.isGameOver()
    return self.observe(), reward, gameOver, self.getState()

In [9]:
# Replay Memory를 class로 정의합니다.
class ReplayMemory:
  def __init__(self, gridSize, maxMemory, discount):
    self.maxMemory = maxMemory
    self.gridSize = gridSize
    self.state_size = self.gridSize * self.gridSize
    self.discount = discount
    canvas = np.zeros((self.gridSize, self.gridSize))
    canvas = np.reshape(canvas, (-1,self.state_size))
    self.inputState = np.empty((self.maxMemory, 100), dtype = np.float32)
    self.actions = np.zeros(self.maxMemory, dtype = np.uint8)
    self.nextState = np.empty((self.maxMemory, 100), dtype = np.float32)
    self.gameOver = np.empty(self.maxMemory, dtype = np.bool)
    self.rewards = np.empty(self.maxMemory, dtype = np.int8) 
    self.count = 0
    self.current = 0

  # 경험을 Replay Memory에 저장합니다.
  def remember(self, currentState, action, reward, nextState, gameOver):
    self.actions[self.current] = action
    self.rewards[self.current] = reward
    self.inputState[self.current, ...] = currentState
    self.nextState[self.current, ...] = nextState
    self.gameOver[self.current] = gameOver
    self.count = max(self.count, self.current + 1)
    self.current = (self.current + 1) % self.maxMemory

  def getBatch(self, y_pred, batch_size, num_actions, state_size, sess, X):
    # 취할 수 있는 가장 큰 배치 사이즈를 선택합니다. (학습 초기에는 batch_size만큼의 기억이 없습니다.)
    memoryLength = self.count
    chosenBatchSize = min(batch_size, memoryLength)

    # 인풋 데이터와 타겟데이터를 선언합니다. 
    inputs = np.zeros((chosenBatchSize, state_size))
    targets = np.zeros((chosenBatchSize, num_actions))

    # 배치안의 값을 설정합니다.
    for i in range(chosenBatchSize):
      # 배치에 포함될 기억을 랜덤으로 선택합니다.
      randomIndex = random.randrange(0, memoryLength)
      # 현재 상태와 Q값을 불러옵니다.
      current_inputState = np.reshape(self.inputState[randomIndex], (1, 100))
      target = sess.run(y_pred, feed_dict={X: current_inputState})
      
      # 현재 상태 바로 다음 상태를 불러오고 다음 상태에서 취할수 있는 가장 큰 Q값을 계산합니다.
      current_nextState = np.reshape(self.nextState[randomIndex], (1, 100))
      nextStateQ = sess.run(y_pred, feed_dict={X: current_nextState})      
      nextStateMaxQ = np.amax(nextStateQ)
      # 만약 게임오버라면 reward로 Q값을 업데이트하고 
      if (self.gameOver[randomIndex] == True):
        target[self.actions[randomIndex]] = self.rewards[randomIndex]
      # 게임오버가 아니라면 타겟 Q값(최적의 Q값)을 아래 수식을 이용해서 계산합니다.
      # Q* = reward + discount(gamma) * max_a' Q(s',a')
      else:
        target[self.actions[randomIndex]] = self.rewards[randomIndex] + self.discount * nextStateMaxQ

      # 인풋과 타겟 데이터에 값을 지정합니다.
      inputs[i] = current_inputState
      targets[i] = target

    return inputs, targets

In [10]:
# 학습을 진행하는 main 함수를 정의합니다.
def main(_):
  print("트레이닝을 시작합니다.")

  # 게임 플레이 환경을 선언합니다.
  env = CatchEnvironment(gridSize)

  # Replay Memory를 선언합니다.
  memory = ReplayMemory(gridSize, maxMemory, discount)

  # 학습된 파라미터를 저장하기 위한 tf.train.Saver를 선언합니다.
  saver = tf.train.Saver()
  
  winCount = 0
  with tf.Session() as sess:   
    # 변수들의 초기값을 할당합니다.
    sess.run(tf.global_variables_initializer())

    for i in range(num_epochs+1):
      # 환경을 초기화합니다.
      err = 0
      env.reset()
     
      isGameOver = False

      # 최초의 상태를 불러옵니다.
      currentState = env.observe()
            
      while (isGameOver != True):
        action = -9999  # Q값을 초기화합니다.
        # epsilon-Greedy 기법에 따라 랜덤한 행동을 할지 최적의 행동을 할지를 결정합니다.
        global epsilon
        if (randf(0, 1) <= epsilon):
          # epsilon 확률만큼 랜덤한 행동을 합니다.
          action = random.randrange(0, num_actions)
        else:          
          # (1-epsilon) 확률만큼 최적의 행동을 합니다.
          # 현재 상태를 DQN의 인풋으로 넣어서 예측된 최적의 Q(s,a)값들을 리턴받습니다.
          q = sess.run(y_pred, feed_dict={x: currentState})   
          # Q(s,a)가 가장 높은 행동을 선택합니다.
          action = q.argmax()

        # epsilon값을 0.9999만큼 Decay합니다.
        if (epsilon > epsilonMinimumValue):
          epsilon = epsilon * 0.999
        
        # 에이전트가 행동을 하고 다음 보상과 다음 상태에 대한 정보를 리턴 받습니다.
        nextState, reward, gameOver, stateInfo = env.act(action)
            
        # 만약 과일을 제대로 받아냈다면 승리 횟수를 1 올립니다.
        if (reward == 1):
          winCount = winCount + 1

        # 에이전트가 수집한 정보를 Replay Memory에 저장합니다.
        memory.remember(currentState, action, reward, nextState, gameOver)
        
        # 현재 상태를 다음 상태로 업데이트하고 GameOver유무를 체크합니다.
        currentState = nextState
        isGameOver = gameOver
                
        # Replay Memory로부터 학습에 사용할 Batch 데이터를 불러옵니다.
        inputs, targets = memory.getBatch(y_pred, batch_size, num_actions, state_size, sess, x)
        
        # 최적화를 수행하고 손실함수를 리턴받습니다.
        _, loss_print = sess.run([optimizer, loss], feed_dict={x: inputs, y: targets})  
        err = err + loss_print

      print("반복(Epoch): %d, 에러(err): %.4f, 승리횟수(Win count): %d, 승리비율(Win ratio): %.4f" % (i, err, winCount, float(winCount)/float(i+1)*100))
    # 학습이 모두 끝나면 파라미터를 지정된 경로에 저장합니다.
    print("트레이닝 완료")
    save_path = saver.save(sess, os.getcwd()+"/model.ckpt")
    print("%s 경로에 파라미터가 저장되었습니다" % save_path)

In [11]:
if __name__ == '__main__':
  # main 함수를 호출합니다.
  tf.app.run()

트레이닝을 시작합니다.
반복(Epoch): 0, 에러(err): 0.0410, 승리횟수(Win count): 0, 승리비율(Win ratio): 0.0000
반복(Epoch): 1, 에러(err): 0.0600, 승리횟수(Win count): 1, 승리비율(Win ratio): 50.0000
반복(Epoch): 2, 에러(err): 0.2094, 승리횟수(Win count): 2, 승리비율(Win ratio): 66.6667
반복(Epoch): 3, 에러(err): 0.2332, 승리횟수(Win count): 2, 승리비율(Win ratio): 50.0000
반복(Epoch): 4, 에러(err): 0.3235, 승리횟수(Win count): 2, 승리비율(Win ratio): 40.0000
반복(Epoch): 5, 에러(err): 0.3279, 승리횟수(Win count): 2, 승리비율(Win ratio): 33.3333
반복(Epoch): 6, 에러(err): 0.3857, 승리횟수(Win count): 3, 승리비율(Win ratio): 42.8571
반복(Epoch): 7, 에러(err): 0.3825, 승리횟수(Win count): 3, 승리비율(Win ratio): 37.5000
반복(Epoch): 8, 에러(err): 0.4670, 승리횟수(Win count): 4, 승리비율(Win ratio): 44.4444
반복(Epoch): 9, 에러(err): 0.4345, 승리횟수(Win count): 4, 승리비율(Win ratio): 40.0000
반복(Epoch): 10, 에러(err): 0.3747, 승리횟수(Win count): 4, 승리비율(Win ratio): 36.3636
반복(Epoch): 11, 에러(err): 0.3997, 승리횟수(Win count): 4, 승리비율(Win ratio): 33.3333
반복(Epoch): 12, 에러(err): 0.4478, 승리횟수(Win count): 4, 승리비율(Win ratio): 30.76

반복(Epoch): 106, 에러(err): 0.2598, 승리횟수(Win count): 30, 승리비율(Win ratio): 28.0374
반복(Epoch): 107, 에러(err): 0.2318, 승리횟수(Win count): 30, 승리비율(Win ratio): 27.7778
반복(Epoch): 108, 에러(err): 0.2442, 승리횟수(Win count): 31, 승리비율(Win ratio): 28.4404
반복(Epoch): 109, 에러(err): 0.2723, 승리횟수(Win count): 31, 승리비율(Win ratio): 28.1818
반복(Epoch): 110, 에러(err): 0.2554, 승리횟수(Win count): 32, 승리비율(Win ratio): 28.8288
반복(Epoch): 111, 에러(err): 0.2675, 승리횟수(Win count): 32, 승리비율(Win ratio): 28.5714
반복(Epoch): 112, 에러(err): 0.2604, 승리횟수(Win count): 32, 승리비율(Win ratio): 28.3186
반복(Epoch): 113, 에러(err): 0.2183, 승리횟수(Win count): 32, 승리비율(Win ratio): 28.0702
반복(Epoch): 114, 에러(err): 0.2105, 승리횟수(Win count): 32, 승리비율(Win ratio): 27.8261
반복(Epoch): 115, 에러(err): 0.2453, 승리횟수(Win count): 33, 승리비율(Win ratio): 28.4483
반복(Epoch): 116, 에러(err): 0.1998, 승리횟수(Win count): 33, 승리비율(Win ratio): 28.2051
반복(Epoch): 117, 에러(err): 0.1994, 승리횟수(Win count): 34, 승리비율(Win ratio): 28.8136
반복(Epoch): 118, 에러(err): 0.1766, 승리횟수(Win count): 34

반복(Epoch): 210, 에러(err): 0.1181, 승리횟수(Win count): 79, 승리비율(Win ratio): 37.4408
반복(Epoch): 211, 에러(err): 0.1632, 승리횟수(Win count): 80, 승리비율(Win ratio): 37.7358
반복(Epoch): 212, 에러(err): 0.1433, 승리횟수(Win count): 81, 승리비율(Win ratio): 38.0282
반복(Epoch): 213, 에러(err): 0.1820, 승리횟수(Win count): 81, 승리비율(Win ratio): 37.8505
반복(Epoch): 214, 에러(err): 0.1771, 승리횟수(Win count): 82, 승리비율(Win ratio): 38.1395
반복(Epoch): 215, 에러(err): 0.1330, 승리횟수(Win count): 83, 승리비율(Win ratio): 38.4259
반복(Epoch): 216, 에러(err): 0.1017, 승리횟수(Win count): 83, 승리비율(Win ratio): 38.2488
반복(Epoch): 217, 에러(err): 0.1300, 승리횟수(Win count): 84, 승리비율(Win ratio): 38.5321
반복(Epoch): 218, 에러(err): 0.1159, 승리횟수(Win count): 85, 승리비율(Win ratio): 38.8128
반복(Epoch): 219, 에러(err): 0.1439, 승리횟수(Win count): 85, 승리비율(Win ratio): 38.6364
반복(Epoch): 220, 에러(err): 0.1155, 승리횟수(Win count): 86, 승리비율(Win ratio): 38.9140
반복(Epoch): 221, 에러(err): 0.1319, 승리횟수(Win count): 87, 승리비율(Win ratio): 39.1892
반복(Epoch): 222, 에러(err): 0.1761, 승리횟수(Win count): 87

반복(Epoch): 313, 에러(err): 0.1143, 승리횟수(Win count): 139, 승리비율(Win ratio): 44.2675
반복(Epoch): 314, 에러(err): 0.0906, 승리횟수(Win count): 140, 승리비율(Win ratio): 44.4444
반복(Epoch): 315, 에러(err): 0.0564, 승리횟수(Win count): 141, 승리비율(Win ratio): 44.6203
반복(Epoch): 316, 에러(err): 0.0715, 승리횟수(Win count): 141, 승리비율(Win ratio): 44.4795
반복(Epoch): 317, 에러(err): 0.0551, 승리횟수(Win count): 142, 승리비율(Win ratio): 44.6541
반복(Epoch): 318, 에러(err): 0.0874, 승리횟수(Win count): 143, 승리비율(Win ratio): 44.8276
반복(Epoch): 319, 에러(err): 0.1059, 승리횟수(Win count): 144, 승리비율(Win ratio): 45.0000
반복(Epoch): 320, 에러(err): 0.1104, 승리횟수(Win count): 145, 승리비율(Win ratio): 45.1713
반복(Epoch): 321, 에러(err): 0.1009, 승리횟수(Win count): 145, 승리비율(Win ratio): 45.0311
반복(Epoch): 322, 에러(err): 0.0613, 승리횟수(Win count): 146, 승리비율(Win ratio): 45.2012
반복(Epoch): 323, 에러(err): 0.1569, 승리횟수(Win count): 147, 승리비율(Win ratio): 45.3704
반복(Epoch): 324, 에러(err): 0.0918, 승리횟수(Win count): 148, 승리비율(Win ratio): 45.5385
반복(Epoch): 325, 에러(err): 0.0968, 승리횟수(Wi

반복(Epoch): 416, 에러(err): 0.0189, 승리횟수(Win count): 234, 승리비율(Win ratio): 56.1151
반복(Epoch): 417, 에러(err): 0.0234, 승리횟수(Win count): 235, 승리비율(Win ratio): 56.2201
반복(Epoch): 418, 에러(err): 0.0111, 승리횟수(Win count): 236, 승리비율(Win ratio): 56.3246
반복(Epoch): 419, 에러(err): 0.0172, 승리횟수(Win count): 237, 승리비율(Win ratio): 56.4286
반복(Epoch): 420, 에러(err): 0.0195, 승리횟수(Win count): 238, 승리비율(Win ratio): 56.5321
반복(Epoch): 421, 에러(err): 0.0122, 승리횟수(Win count): 238, 승리비율(Win ratio): 56.3981
반복(Epoch): 422, 에러(err): 0.0164, 승리횟수(Win count): 239, 승리비율(Win ratio): 56.5012
반복(Epoch): 423, 에러(err): 0.0523, 승리횟수(Win count): 240, 승리비율(Win ratio): 56.6038
반복(Epoch): 424, 에러(err): 0.0231, 승리횟수(Win count): 241, 승리비율(Win ratio): 56.7059
반복(Epoch): 425, 에러(err): 0.0163, 승리횟수(Win count): 242, 승리비율(Win ratio): 56.8075
반복(Epoch): 426, 에러(err): 0.0154, 승리횟수(Win count): 243, 승리비율(Win ratio): 56.9087
반복(Epoch): 427, 에러(err): 0.0433, 승리횟수(Win count): 244, 승리비율(Win ratio): 57.0093
반복(Epoch): 428, 에러(err): 0.0200, 승리횟수(Wi

...

반복(Epoch): 1736, 에러(err): 0.0000, 승리횟수(Win count): 1548, 승리비율(Win ratio): 89.1192
반복(Epoch): 1737, 에러(err): 0.0000, 승리횟수(Win count): 1549, 승리비율(Win ratio): 89.1254
반복(Epoch): 1738, 에러(err): 0.0000, 승리횟수(Win count): 1550, 승리비율(Win ratio): 89.1317
반복(Epoch): 1739, 에러(err): 0.0000, 승리횟수(Win count): 1551, 승리비율(Win ratio): 89.1379
반복(Epoch): 1740, 에러(err): 0.0000, 승리횟수(Win count): 1552, 승리비율(Win ratio): 89.1442
반복(Epoch): 1741, 에러(err): 0.0000, 승리횟수(Win count): 1553, 승리비율(Win ratio): 89.1504
반복(Epoch): 1742, 에러(err): 0.0000, 승리횟수(Win count): 1554, 승리비율(Win ratio): 89.1566
반복(Epoch): 1743, 에러(err): 0.0000, 승리횟수(Win count): 1555, 승리비율(Win ratio): 89.1628
반복(Epoch): 1744, 에러(err): 0.0000, 승리횟수(Win count): 1556, 승리비율(Win ratio): 89.1691
반복(Epoch): 1745, 에러(err): 0.0000, 승리횟수(Win count): 1557, 승리비율(Win ratio): 89.1753
반복(Epoch): 1746, 에러(err): 0.0000, 승리횟수(Win count): 1558, 승리비율(Win ratio): 89.1815
반복(Epoch): 1747, 에러(err): 0.0000, 승리횟수(Win count): 1559, 승리비율(Win ratio): 89.1876
반복(Epoch): 1748,

반복(Epoch): 1836, 에러(err): 0.0000, 승리횟수(Win count): 1648, 승리비율(Win ratio): 89.7115
반복(Epoch): 1837, 에러(err): 0.0000, 승리횟수(Win count): 1649, 승리비율(Win ratio): 89.7171
반복(Epoch): 1838, 에러(err): 0.0000, 승리횟수(Win count): 1650, 승리비율(Win ratio): 89.7227
반복(Epoch): 1839, 에러(err): 0.0000, 승리횟수(Win count): 1651, 승리비율(Win ratio): 89.7283
반복(Epoch): 1840, 에러(err): 0.0000, 승리횟수(Win count): 1652, 승리비율(Win ratio): 89.7338
반복(Epoch): 1841, 에러(err): 0.0000, 승리횟수(Win count): 1653, 승리비율(Win ratio): 89.7394
반복(Epoch): 1842, 에러(err): 0.0000, 승리횟수(Win count): 1654, 승리비율(Win ratio): 89.7450
반복(Epoch): 1843, 에러(err): 0.0000, 승리횟수(Win count): 1655, 승리비율(Win ratio): 89.7505
반복(Epoch): 1844, 에러(err): 0.0000, 승리횟수(Win count): 1656, 승리비율(Win ratio): 89.7561
반복(Epoch): 1845, 에러(err): 0.0000, 승리횟수(Win count): 1657, 승리비율(Win ratio): 89.7616
반복(Epoch): 1846, 에러(err): 0.0000, 승리횟수(Win count): 1658, 승리비율(Win ratio): 89.7672
반복(Epoch): 1847, 에러(err): 0.0000, 승리횟수(Win count): 1659, 승리비율(Win ratio): 89.7727
반복(Epoch): 1848,

반복(Epoch): 1936, 에러(err): 0.0003, 승리횟수(Win count): 1748, 승리비율(Win ratio): 90.2426
반복(Epoch): 1937, 에러(err): 0.0003, 승리횟수(Win count): 1749, 승리비율(Win ratio): 90.2477
반복(Epoch): 1938, 에러(err): 0.0001, 승리횟수(Win count): 1750, 승리비율(Win ratio): 90.2527
반복(Epoch): 1939, 에러(err): 0.0005, 승리횟수(Win count): 1751, 승리비율(Win ratio): 90.2577
반복(Epoch): 1940, 에러(err): 0.0001, 승리횟수(Win count): 1752, 승리비율(Win ratio): 90.2628
반복(Epoch): 1941, 에러(err): 0.0000, 승리횟수(Win count): 1753, 승리비율(Win ratio): 90.2678
반복(Epoch): 1942, 에러(err): 0.0003, 승리횟수(Win count): 1754, 승리비율(Win ratio): 90.2728
반복(Epoch): 1943, 에러(err): 0.0001, 승리횟수(Win count): 1755, 승리비율(Win ratio): 90.2778
반복(Epoch): 1944, 에러(err): 0.0000, 승리횟수(Win count): 1756, 승리비율(Win ratio): 90.2828
반복(Epoch): 1945, 에러(err): 0.0000, 승리횟수(Win count): 1757, 승리비율(Win ratio): 90.2878
반복(Epoch): 1946, 에러(err): 0.0005, 승리횟수(Win count): 1758, 승리비율(Win ratio): 90.2928
반복(Epoch): 1947, 에러(err): 0.0003, 승리횟수(Win count): 1759, 승리비율(Win ratio): 90.2977
반복(Epoch): 1948,

SystemExit: 

C:\Users\pnu\AppData\Roaming\Python\Python38\site-packages\IPython\core\interactiveshell.py:3445: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
